# Operator splitting techniques

The operator splitting strategy for ODEs will be experimented on the Oregonator system of equations.
The system of ordinary differential equations reads:

\begin{eqnarray}
{\mathrm d}_\tau y_1 &=&y_2-y_1,\\[1ex]
\epsilon\,  {\mathrm d}_\tau y_2 &=&q\,y_3-y_3\,y_2+y_2(1-y_2),\\[1ex]
\mu\,  {\mathrm d}_\tau y_3 &=&-q\,y_3- y_3\,y_2+f\, y_1,\\[1ex]
\end{eqnarray}

with paremeters

\begin{equation} 
\epsilon = 10^{-2},\quad \mu =10^{-6}, \quad f=3,\qquad q=2.10^{-4}.
\end{equation}

In general $\mu \ll \epsilon \ll 1$ and $q \ll 1$.

In [ ]:
import numpy as np

from scipy.integrate import solve_ivp

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.models import PrintfTickFormatter

from mylib.model import oregonator_model
import mylib.integration as integration

import time

output_notebook(hide_banner=True)

## Quasi exact solution and singular perturbation

The quasi-exact solution is obtained by using an implicit Runge-Kutta method of Radau IIA family of order 5 with stepsize control and fine tolerances.

### Python version of radau5

In [ ]:
def plot_quasi_exact_sol():
    
    eps = 1.e-2
    mu = 1.e-4
    f = 1.
    q = 2.e-4
    
    om = oregonator_model(eps=eps, mu=mu, f=f, q=q)
    fcn = om.fcn
    jac = om.jac
    fcn_ps= om.fcn_ps
    
    tini = 0. 
    tend = 30.
    
    yini = (0.5, 1.e-3, (f*0.5)/(q+1.e-3))
    yini_ps = (0.5, 1.e-3)
    
    sol = solve_ivp(fcn, (tini, tend), yini, method="Radau", rtol=1.e-12, atol=1.e-12, jac=jac)
    sol_ps = solve_ivp(fcn_ps, (tini, tend), yini_ps, method="Radau", rtol=1.e-12, atol=1.e-12)
    
    fig_sol_y1 = figure(x_range=(tini, tend), width=950, height=300, 
                        title="Solution y1 (click on legend to hide corresponding plot)")
    fig_sol_y1.line(sol.t, sol.y[0], legend="y1", line_width=2)
    fig_sol_y1.line(sol_ps.t, sol_ps.y[0], legend="y1 (singular perturbation)", 
                    color="black", line_width=2, line_dash="dotted")
    fig_sol_y1.legend.click_policy="hide"
    fig_sol_y1.legend.location = "top_left"

    fig_sol_y2 = figure(x_range=(tini, tend), width=950, height=300, 
                        title="Solution y2 (click on legend to hide corresponding plot)")
    fig_sol_y2.line(sol.t, sol.y[1], legend="y2", line_width=2)    
    fig_sol_y2.line(sol_ps.t, sol_ps.y[1], legend="y2 (singular pert.)",
                    color="black", line_width=2, line_dash="dotted")
    fig_sol_y2.legend.click_policy="hide"
    fig_sol_y2.legend.location = "top_left"

    fig_sol_y3 = figure(x_range=(tini, tend), width=950, height=300, 
                        title="Solution y3 (click on legend to hide corresponding plot)")
    fig_sol_y3.line(sol.t, sol.y[2], legend="y3", line_width=2)    
    fig_sol_y3.line(sol_ps.t, (f*sol_ps.y[0])/(q+sol_ps.y[1]), legend="(f.y1)/(q+y2) (singular perturbation)",
                    color="black", line_width=2, line_dash="dotted")
    fig_sol_y3.line(sol.t, (f*sol.y[0])/(q+sol.y[1]), legend="(f.y1)/(q+y2)",
                    color="red", line_width=2, line_dash="dotted")

    fig_sol_y3.legend.click_policy="hide"
    fig_sol_y3.legend.location = "top_right"

    show(column(fig_sol_y1, fig_sol_y2, fig_sol_y3))   
    
plot_quasi_exact_sol()

### Fortran version of Radau5

In [ ]:
def plot_quasi_exact_sol_fortran():
    
    eps = 1.e-2
    mu = 1.e-4
    f = 1.
    q = 2.e-4
    
    om = oregonator_model(eps=eps, mu=mu, f=f, q=q)
    fcn_radau = om.fcn_radau
    fcn_ps_radau = om.fcn_ps_radau
    
    tini = 0. 
    tend = 30.
    
    yini = (0.5, 1.e-1, (f*0.5)/(q+1.e-1))
    yini_ps = (0.5, 1.e-1)
    
    sol = integration.radau5(tini, tend, yini, fcn_radau, njac=3, rtol=1.e-12, atol=1.e-12)
    sol_ps = integration.radau5(tini, tend, yini_ps, fcn_ps_radau, njac=2, rtol=1.e-12, atol=1.e-12)
    
    fig_sol_y1 = figure(x_range=(tini, tend), width=950, height=300, 
                        title="Solution y1 (click on legend to hide corresponding plot)")
    fig_sol_y1.line(sol.t, sol.y[0], legend="y1", line_width=2)
    fig_sol_y1.line(sol_ps.t, sol_ps.y[0], legend="y1 (singular perturbation)", 
                    color="black", line_width=2, line_dash="dotted")
    fig_sol_y1.legend.click_policy="hide"
    fig_sol_y1.legend.location = "top_left"

    fig_sol_y2 = figure(x_range=(tini, tend), width=950, height=300, 
                        title="Solution y2 (click on legend to hide corresponding plot)")
    fig_sol_y2.line(sol.t, sol.y[1], legend="y2", line_width=2)    
    fig_sol_y2.line(sol_ps.t, sol_ps.y[1], legend="y2 (singular pert.)",
                    color="black", line_width=2, line_dash="dotted")
    fig_sol_y2.legend.click_policy="hide"
    fig_sol_y2.legend.location = "top_left"

    fig_sol_y3 = figure(x_range=(tini, tend), width=950, height=300, 
                        title="Solution y3 (click on legend to hide corresponding plot)")
    fig_sol_y3.line(sol.t, sol.y[2], legend="y3", line_width=2)    
    fig_sol_y3.line(sol_ps.t, (f*sol_ps.y[0])/(q+sol_ps.y[1]), legend="(f.y1)/(q+y2) (singular perturbation)",
                    color="black", line_width=2, line_dash="dotted")
    fig_sol_y3.line(sol.t, (f*sol.y[0])/(q+sol.y[1]), legend="(f.y1)/(q+y2)",
                    color="red", line_width=2, line_dash="dotted")
    fig_sol_y3.legend.click_policy="hide"
    fig_sol_y3.legend.location = "top_right"

    show(column(fig_sol_y1, fig_sol_y2, fig_sol_y3))
    
plot_quasi_exact_sol_fortran()

## Operator splitting techniques

### First form with Lie splitting

In [ ]:
def plot_lie_splitting_01():
    
    eps = 1.e-2
    mu = 1.e-4
    f = 1.
    q = 2.e-4
        
    tini = 0. 
    tend = 30.
    
    yini = (0.5, 1.e-3, (f*0.5)/(q+1.e-3))
    
    om = oregonator_model(eps=eps, mu=mu, f=f, q=q)
    fcn_radau = om.fcn_radau

    nt = 3001
    t = np.linspace(tini, tend, nt)
    dt = (tend-tini) / (nt-1)
    
    y = np.zeros((3, nt), order='F')
    y[:,0] = yini
    y_exa = np.zeros((3, nt), order='F')
    y_exa[:,0] = yini
    
    for it, tn in enumerate(t[:-1]):
        
        # operator A
        om_a = oregonator_model(eps=eps, mu=mu, f=f, q=q, y30=y[2,it])
        fcn_a_radau = om_a.fcn_a1_radau
        yini_a = (y[0, it] , y[1,it])
        sol_y_a = integration.radau5(tn, tn+dt, yini_a, fcn_a_radau, njac=2, rtol=1.e-12, atol=1.e-12)
        y[0, it+1] = sol_y_a.y[0,-1]
        y[1, it+1] = sol_y_a.y[1,-1]

        # operator B
        om_b = oregonator_model(eps=eps, mu=mu, f=f, q=q, y10=sol_y_a.y[0,-1], y20=sol_y_a.y[1,-1])
        fcn_b_radau = om_b.fcn_b1_radau
        yini_b = (y[2,it],)
        sol_y_b = integration.radau5(tn, tn+dt, yini_b, fcn_b_radau, njac=1, rtol=1.e-12, atol=1.e-12)
        y[2,it+1] = sol_y_b.y[0,-1]
  
        # quasi exact sol 
        sol_exa = integration.radau5(tn, tn+dt, y_exa[:,it], fcn_radau, njac=3, rtol=1.e-12, atol=1.e-12)
        y_exa[:,it+1] = sol_exa.y[:,-1]
        
    fig_sol_y1 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y1")
    fig_sol_y1.x(t, y[0], legend="y1", line_width=2)
    fig_sol_y1.line(t, y_exa[0], legend="quasi exact y1", color="black")
    fig_sol_y1.legend.click_policy="hide"
    fig_sol_y2 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y2")
    fig_sol_y2.x(t, y[1], legend="y2", line_width=2)
    fig_sol_y2.line(t, y_exa[1], legend="quasi exact y1", color="black")
    fig_sol_y2.legend.click_policy="hide"
    fig_sol_y3 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y3")
    fig_sol_y3.x(t, y[2], legend="y3", line_width=2)    
    fig_sol_y3.line(t, y_exa[2], legend="quasi exact y3", color="black")
    fig_sol_y3.line(t, (f*y[0])/(q+y[1]) - y[2], legend="yeq3 - y3", color="red")
    fig_sol_y3.legend.click_policy="hide"

    show(column(fig_sol_y1, fig_sol_y2, fig_sol_y3))
    
    err_y1 = 0.
    err_y2 = 0.
    err_y3 = 0.
    for it, tn in enumerate(t):
        err_y1 += dt * (y_exa[0, it] - y[0, it]) * (y_exa[0, it] - y[0, it])
        err_y2 += dt * (y_exa[1, it] - y[1, it]) * (y_exa[1, it] - y[1, it])
        err_y3 += dt * (y_exa[2, it] - y[2, it]) * (y_exa[2, it] - y[2, it])
    err_y1 = np.sqrt(err_y1/tend-tini) / np.max(y_exa[0, :])
    err_y2 = np.sqrt(err_y2/tend-tini) / np.max(y_exa[1, :])
    err_y3 = np.sqrt(err_y3/tend-tini) / np.max(y_exa[2, :])
    print(f"Relative norm l2 of error for y1 : {err_y1:e}")
    print(f"Relative norm l2 of error for y2 : {err_y2:e}")
    print(f"Relative norm l2 of error for y3 : {err_y3:e}")
    
plot_lie_splitting_01()

### First form with Strang splitting

In [ ]:
def plot_strang_splitting_01():
    
    eps = 1.e-2
    mu = 1.e-6
    f = 1.
    q = 2.e-4
        
    tini = 0. 
    tend = 30.
    
    yini = (0.5, 1.e-3, (f*0.5)/(q+1.e-3))
    
    om = oregonator_model(eps=eps, mu=mu, f=f, q=q)
    fcn_radau = om.fcn_radau

    nt = 3001
    t = np.linspace(tini, tend, nt)
    dt = (tend-tini) / (nt-1)
    
    y = np.zeros((3, nt), order='F')
    y[:,0] = yini
    y_exa = np.zeros((3, nt), order='F')
    y_exa[:,0] = yini

    for it, tn in enumerate(t[:-1]):
                
        # operator B
        om_b = oregonator_model(eps=eps, mu=mu, f=f, q=q, y10=y[0,it], y20=y[1,it])
        fcn_b_radau = om_b.fcn_b1_radau
        yini_b = (y[2,it],)
        sol_y_b = integration.radau5(tn, tn+dt/2, yini_b, fcn_b_radau, njac=1, rtol=1.e-12, atol=1.e-12)
      
        # operator A
        om_a = oregonator_model(eps=eps, mu=mu, f=f, q=q, y30=sol_y_b.y[0,-1])
        fcn_a_radau = om_a.fcn_a1_radau
        yini_a = (y[0, it] , y[1,it])
        sol_y_a = integration.radau5(tn, tn+dt, yini_a, fcn_a_radau, njac=2, rtol=1.e-12, atol=1.e-12)
        y[0, it+1] = sol_y_a.y[0,-1]
        y[1, it+1] = sol_y_a.y[1,-1]

        # operator B
        om_b = oregonator_model(eps=eps, mu=mu, f=f, q=q, y10=sol_y_a.y[0,-1], y20=sol_y_a.y[1,-1])
        fcn_b_radau = om_b.fcn_b1_radau
        yini_b = (sol_y_b.y[0,-1],)
        sol_y_b = integration.radau5(tn+dt/2, tn+dt, yini_b, fcn_b_radau, njac=1, rtol=1.e-12, atol=1.e-12)
        y[2,it+1] = sol_y_b.y[0,-1]
        
        # quasi exact sol 
        sol_exa = integration.radau5(tn, tn+dt, y_exa[:,it], fcn_radau, njac=3, rtol=1.e-12, atol=1.e-12)
        y_exa[:,it+1] = sol_exa.y[:,-1]

    fig_sol_y1 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y1")
    fig_sol_y1.x(t, y[0], legend="y1", line_width=2)
    fig_sol_y1.line(t, y_exa[0], legend="quasi exact y1", color="black")
    fig_sol_y1.legend.click_policy="hide"
    fig_sol_y2 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y2")
    fig_sol_y2.x(t, y[1], legend="y2", line_width=2)
    fig_sol_y2.line(t, y_exa[1], legend="quasi exact y1", color="black")
    fig_sol_y2.legend.click_policy="hide"
    fig_sol_y3 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y3")
    fig_sol_y3.x(t, y[2], legend="y3", line_width=2)    
    fig_sol_y3.line(t, y_exa[2], legend="quasi exact y3", color="black")
    fig_sol_y3.line(t, (f*y[0])/(q+y[1]) - y[2], legend="yeq3 - y3", color="red")
    fig_sol_y3.legend.click_policy="hide"

    show(column(fig_sol_y1, fig_sol_y2, fig_sol_y3))
    
    err_y1 = 0.
    err_y2 = 0.
    err_y3 = 0.
    for it, tn in enumerate(t):
        err_y1 += dt * (y_exa[0, it] - y[0, it]) * (y_exa[0, it] - y[0, it])
        err_y2 += dt * (y_exa[1, it] - y[1, it]) * (y_exa[1, it] - y[1, it])
        err_y3 += dt * (y_exa[2, it] - y[2, it]) * (y_exa[2, it] - y[2, it])
    err_y1 = np.sqrt(err_y1/tend-tini) / np.max(y_exa[0, :])
    err_y2 = np.sqrt(err_y2/tend-tini) / np.max(y_exa[1, :])
    err_y3 = np.sqrt(err_y3/tend-tini) / np.max(y_exa[2, :])
    print(f"Relative norm l2 of error for y1 : {err_y1:e}")
    print(f"Relative norm l2 of error for y2 : {err_y2:e}")
    print(f"Relative norm l2 of error for y3 : {err_y3:e}")

plot_strang_splitting_01()

### Second form with Lie splitting

In [ ]:
def plot_lie_splitting_02():
    
    eps = 1.e-2
    mu = 1.e-4
    f = 1.
    q = 2.e-4
        
    tini = 0. 
    tend = 30.
    
    yini = (0.5, 1.e-3, (f*0.5)/(q+1.e-3))
    
    om = oregonator_model(eps=eps, mu=mu, f=f, q=q)
    fcn_radau = om.fcn_radau

    nt = 3001
    t = np.linspace(tini, tend, nt)
    dt = (tend-tini) / (nt-1)
    
    y = np.zeros((3, nt), order='F')
    y[:,0] = yini
    y_exa = np.zeros((3, nt), order='F')
    y_exa[:,0] = yini

    
    for it, tn in enumerate(t[:-1]):
        
        # operator A
        om_a = oregonator_model(eps=eps, mu=mu, f=f, q=q, y30=y[2,it])
        fcn_a_radau = om_a.fcn_a2_radau
        yini_a = (y[0, it] , y[1,it])
        sol_y_a = integration.radau5(tn, tn+dt, yini_a, fcn_a_radau, njac=2, rtol=1.e-12, atol=1.e-12)
        y[0, it+1] = sol_y_a.y[0,-1]

        # operator B
        om_b = oregonator_model(eps=eps, mu=mu, f=f, q=q, y10=sol_y_a.y[0,-1])
        fcn_b_radau = om_b.fcn_b2_radau
        yini_b = (sol_y_a.y[1,-1], y[2,it])
        sol_y_b = integration.radau5(tn, tn+dt, yini_b, fcn_b_radau, njac=1, rtol=1.e-12, atol=1.e-12)
        y[1,it+1] = sol_y_b.y[0,-1]
        y[2,it+1] = sol_y_b.y[1,-1]
        
        # quasi exact sol 
        sol_exa = integration.radau5(tn, tn+dt, y_exa[:,it], fcn_radau, njac=3, rtol=1.e-12, atol=1.e-12)
        y_exa[:,it+1] = sol_exa.y[:,-1]

        
    fig_sol_y1 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y1")
    fig_sol_y1.x(t, y[0], legend="y1", line_width=2)
    fig_sol_y1.line(t, y_exa[0], legend="quasi exact y1", color="black")
    fig_sol_y1.legend.click_policy="hide"
    fig_sol_y2 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y2")
    fig_sol_y2.x(t, y[1], legend="y2", line_width=2)
    fig_sol_y2.line(t, y_exa[1], legend="quasi exact y1", color="black")
    fig_sol_y2.legend.click_policy="hide"
    fig_sol_y3 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y3")
    fig_sol_y3.x(t, y[2], legend="y3", line_width=2)    
    fig_sol_y3.line(t, y_exa[2], legend="quasi exact y3", color="black")
    fig_sol_y3.line(t, (f*y[0])/(q+y[1]) - y[2], legend="yeq3 - y3", color="red")
    fig_sol_y3.legend.click_policy="hide"

    show(column(fig_sol_y1, fig_sol_y2, fig_sol_y3))
    
    err_y1 = 0.
    err_y2 = 0.
    err_y3 = 0.
    for it, tn in enumerate(t):
        err_y1 += dt * (y_exa[0, it] - y[0, it]) * (y_exa[0, it] - y[0, it])
        err_y2 += dt * (y_exa[1, it] - y[1, it]) * (y_exa[1, it] - y[1, it])
        err_y3 += dt * (y_exa[2, it] - y[2, it]) * (y_exa[2, it] - y[2, it])
    err_y1 = np.sqrt(err_y1/tend-tini) / np.max(y_exa[0, :])
    err_y2 = np.sqrt(err_y2/tend-tini) / np.max(y_exa[1, :])
    err_y3 = np.sqrt(err_y3/tend-tini) / np.max(y_exa[2, :])
    print(f"Relative norm l2 of error for y1 : {err_y1:e}")
    print(f"Relative norm l2 of error for y2 : {err_y2:e}")
    print(f"Relative norm l2 of error for y3 : {err_y3:e}")

plot_lie_splitting_02()

### Second form with Strang splitting

In [ ]:
def plot_strang_splitting_02():
    
    eps = 1.e-2
    mu = 1.e-4
    f = 1.
    q = 2.e-4
        
    tini = 0. 
    tend = 30.
    
    yini = (0.5, 1.e-3, (f*0.5)/(q+1.e-3))
    
    om = oregonator_model(eps=eps, mu=mu, f=f, q=q)
    fcn_radau = om.fcn_radau

    nt = 3001
    t = np.linspace(tini, tend, nt)
    dt = (tend-tini) / (nt-1)
    
    y = np.zeros((3, nt), order='F')
    y[:,0] = yini
    y_exa = np.zeros((3, nt), order='F')
    y_exa[:,0] = yini

    
    for it, tn in enumerate(t[:-1]):
                        
        # operator B
        om_b = oregonator_model(eps=eps, mu=mu, f=f, q=q, y10=y[0,it])
        fcn_b_radau = om_b.fcn_b2_radau
        yini_b = (y[1,it], y[2,it],)
        sol_y_b = integration.radau5(tn, tn+dt/2, yini_b, fcn_b_radau, njac=1, rtol=1.e-12, atol=1.e-12)
      
        # operator A
        om_a = oregonator_model(eps=eps, mu=mu, f=f, q=q)
        fcn_a_radau = om_a.fcn_a2_radau
        yini_a = (y[0, it] , sol_y_b.y[0,-1])
        sol_y_a = integration.radau5(tn, tn+dt, yini_a, fcn_a_radau, njac=2, rtol=1.e-12, atol=1.e-12)
        y[0, it+1] = sol_y_a.y[0,-1]

        # operator B
        om_b = oregonator_model(eps=eps, mu=mu, f=f, q=q, y10=sol_y_a.y[0,-1])
        fcn_b_radau = om_b.fcn_b2_radau
        yini_b = (sol_y_a.y[1,-1], sol_y_b.y[1,-1],)
        sol_y_b = integration.radau5(tn+dt/2, tn+dt, yini_b, fcn_b_radau, njac=1, rtol=1.e-12, atol=1.e-12)
        y[1, it+1] = sol_y_b.y[0,-1]
        y[2, it+1] = sol_y_b.y[1,-1]
        
        # quasi exact sol 
        sol_exa = integration.radau5(tn, tn+dt, y_exa[:,it], fcn_radau, njac=3, rtol=1.e-12, atol=1.e-12)
        y_exa[:,it+1] = sol_exa.y[:,-1]

        
    fig_sol_y1 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y1")
    fig_sol_y1.x(t, y[0], legend="y1", line_width=2)
    fig_sol_y1.line(t, y_exa[0], legend="quasi exact y1", color="black")
    fig_sol_y1.legend.click_policy="hide"
    fig_sol_y2 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y2")
    fig_sol_y2.x(t, y[1], legend="y2", line_width=2)
    fig_sol_y2.line(t, y_exa[1], legend="quasi exact y1", color="black")
    fig_sol_y2.legend.click_policy="hide"
    fig_sol_y3 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y3")
    fig_sol_y3.x(t, y[2], legend="y3", line_width=2)    
    fig_sol_y3.line(t, y_exa[2], legend="quasi exact y3", color="black")
    fig_sol_y3.line(t, (f*y[0])/(q+y[1]) - y[2], legend="yeq3 - y3", color="red")
    fig_sol_y3.legend.click_policy="hide"

    show(column(fig_sol_y1, fig_sol_y2, fig_sol_y3))
    
    err_y1 = 0.
    err_y2 = 0.
    err_y3 = 0.
    for it, tn in enumerate(t):
        err_y1 += dt * (y_exa[0, it] - y[0, it]) * (y_exa[0, it] - y[0, it])
        err_y2 += dt * (y_exa[1, it] - y[1, it]) * (y_exa[1, it] - y[1, it])
        err_y3 += dt * (y_exa[2, it] - y[2, it]) * (y_exa[2, it] - y[2, it])
    err_y1 = np.sqrt(err_y1/tend-tini) / np.max(y_exa[0, :])
    err_y2 = np.sqrt(err_y2/tend-tini) / np.max(y_exa[1, :])
    err_y3 = np.sqrt(err_y3/tend-tini) / np.max(y_exa[2, :])
    print(f"Relative norm l2 of error for y1 : {err_y1:e}")
    print(f"Relative norm l2 of error for y2 : {err_y2:e}")
    print(f"Relative norm l2 of error for y3 : {err_y3:e}")

plot_strang_splitting_02()